In [845]:
import numpy as np
import itertools
import random

# 3.1

Напишем функцию формирования проверочной матрицы кода Хэмминга

In [846]:
def generate_hamming_h_matrix(r: int) -> np.ndarray:
    n = 2 ** r - 1  # Число строк и столбцов
    res = []
    cur_r = r - 1
    for i in range(n, 0, -1):
        if i != 2 ** cur_r:
            res.append(list(map(int, f"{i:0{r}b}")))
        else:
            cur_r -= 1

    # Единичная матрица r x r в нижней части
    identity_matrix = np.eye(r, dtype=int)

    # Объединяем верхнюю часть и единичную матрицу
    H = np.vstack((res, identity_matrix))

    return H

Возьмем r = 2, получим проверочную матрицу следующего вида

In [847]:
r = 2
H = generate_hamming_h_matrix(r)
H

array([[1, 1],
       [1, 0],
       [0, 1]])

Функция для построения порождающей матрицы кода Хэмминга на основе проверочной матрицы

In [848]:
def H_to_G(H: np.ndarray, r) -> np.ndarray:
    k = 2 ** r - r - 1
    res = np.eye(k, dtype=int)  # Единичная матрица
    G = np.hstack((res, H[:k]))
    return G

Проверим функцию и построим порождающую матрицу

In [849]:
G = H_to_G(H, r)
G

array([[1, 1, 1]])

Функция для построения таблицы синдромов

In [850]:
def generate_syndrome_table(matrix: np.ndarray, error_weight: int) -> dict:
    n = matrix.shape[0]
    syndrome_table = {}
    for error in range(1, error_weight + 1):
        for error_indices in itertools.combinations(range(n), error):
            error_vector = np.zeros(n, dtype=int)
            for index in error_indices:
                error_vector[index] = 1
            syndrome = error_vector @ matrix % 2
            syndrome_table[tuple(map(int, syndrome))] = tuple(error_indices)

    return syndrome_table

# 3.2

Сформируем таблицу синдромов для всех единичных ошибок

In [851]:
syndrome_table = generate_syndrome_table(H, 1)
syndrome_table

{(1, 1): (0,), (1, 0): (1,), (0, 1): (2,)}

Функция для допущения и проверки ошибки

In [852]:
def hamming_correction_test(G: np.ndarray, H: np.ndarray, syndrome_table: dict, error_degree: int, u: np.ndarray):
    # Шаг 1: Выбираем случайное кодовое слово из G
    print("Кодовое слово (u):", u)
    
    # Шаг 2: Генерируем кодовое слово
    v = u @ G % 2
    print("Отправленное кодовое слово (v):", v)
    
    # Шаг 3: Допускаем ошибку error_degree в принятом кодовом слове
    error = np.zeros(v.shape[0], dtype=int)
    error_indices = random.sample(range(v.shape[0]), error_degree)  # случайные индексы ошибок
    for index in error_indices:
        error[index] = 1
    print("Допущенная ошибка:", error)
    
    # Принятое слово с ошибкой
    received_v = (v + error) % 2
    print("Принятое с ошибкой слово:", received_v)
    
    # Шаг 4: Вычисляем синдром принятого слова
    syndrome = received_v @ H % 2
    print("Синдром принятого сообщения:", syndrome)
    if sum(syndrome) != 0:
        print("Обнаружена ошибка!")
    
    # Шаг 5: Проверяем синдром в таблице синдромов и корректируем ошибку
    if tuple(syndrome) in syndrome_table:
        correction_indices = syndrome_table[tuple(syndrome)]
        for index in correction_indices:
            received_v[index] = (received_v[index] + 1) % 2  # корректируем ошибку
        print("Исправленное сообщение:", received_v)
        
        # Проверка совпадения с отправленным сообщением
        if np.array_equal(v, received_v):
            print("Ошибка была исправлена успешно!")
        else:
            print("Ошибка не была исправлена корректно.")
    else:
        print("Синдрома нет в таблице, ошибка не исправлена.")

Проведем исследование кода Хэмминга для ошибки кратности 1. Передаем сообщение длины 2^r - r - 1 = 2^2 - 2 - 1 = 1: "1 0 0 1".

In [853]:
hamming_correction_test(G, H, syndrome_table, 1, np.array([1]))

Кодовое слово (u): [1]
Отправленное кодовое слово (v): [1 1 1]
Допущенная ошибка: [1 0 0]
Принятое с ошибкой слово: [0 1 1]
Синдром принятого сообщения: [1 1]
Обнаружена ошибка!
Исправленное сообщение: [1 1 1]
Ошибка была исправлена успешно!


Код успешно отловил ошибку и исправил ее. Теперь попробуем допустить двухкратную ошибку

In [854]:
hamming_correction_test(G, H, syndrome_table, 2, np.array([1]))

Кодовое слово (u): [1]
Отправленное кодовое слово (v): [1 1 1]
Допущенная ошибка: [1 1 0]
Принятое с ошибкой слово: [0 0 1]
Синдром принятого сообщения: [0 1]
Обнаружена ошибка!
Исправленное сообщение: [0 0 0]
Ошибка не была исправлена корректно.


Так как код Хэмминга предназначен либо для исправления однократных ошибок, либо для обнаружения двухкратных, мы смогли обнаружить ошибку, но не смогли ее исправить. Попробуем теперь трехкратную ошибку.

In [855]:
hamming_correction_test(G, H, syndrome_table, 3, np.array([1]))

Кодовое слово (u): [1]
Отправленное кодовое слово (v): [1 1 1]
Допущенная ошибка: [1 1 1]
Принятое с ошибкой слово: [0 0 0]
Синдром принятого сообщения: [0 0]
Синдрома нет в таблице, ошибка не исправлена.


Здесь возможно два варианта:
1) Синдрома в таблице синдромов для конкретной трехкратной ошибки нет. Ошибку в данном случае исправить не получится. 
2) Синдром в таблице имеется, программа попытается исправить ошибку, но, конечно, результат будет неверным.

Аналогично, проведем исследование для r = 3. Пересчитаем проверочную матрицу

In [856]:
r = 3
H = generate_hamming_h_matrix(r)
H

array([[1, 1, 1],
       [1, 1, 0],
       [1, 0, 1],
       [0, 1, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

Пересчитаем G

In [857]:
G = H_to_G(H, r)
G

array([[1, 0, 0, 0, 1, 1, 1],
       [0, 1, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 1, 1]])

И составим заново таблицу синдромов

In [858]:
syndrome_table = generate_syndrome_table(H, 1)
syndrome_table

{(1, 1, 1): (0,),
 (1, 1, 0): (1,),
 (1, 0, 1): (2,),
 (0, 1, 1): (3,),
 (1, 0, 0): (4,),
 (0, 1, 0): (5,),
 (0, 0, 1): (6,)}

Передаем слово длины 4: "1 0 0 1". Сначала допустим однократную ошибку

In [859]:
hamming_correction_test(G, H, syndrome_table, 1, np.array([1, 0, 0, 1]))

Кодовое слово (u): [1 0 0 1]
Отправленное кодовое слово (v): [1 0 0 1 1 0 0]
Допущенная ошибка: [0 0 0 1 0 0 0]
Принятое с ошибкой слово: [1 0 0 0 1 0 0]
Синдром принятого сообщения: [0 1 1]
Обнаружена ошибка!
Исправленное сообщение: [1 0 0 1 1 0 0]
Ошибка была исправлена успешно!


Теперь для двухкратной ошибки

In [860]:
hamming_correction_test(G, H, syndrome_table, 2, np.array([1, 0, 0, 1]))

Кодовое слово (u): [1 0 0 1]
Отправленное кодовое слово (v): [1 0 0 1 1 0 0]
Допущенная ошибка: [1 0 1 0 0 0 0]
Принятое с ошибкой слово: [0 0 1 1 1 0 0]
Синдром принятого сообщения: [0 1 0]
Обнаружена ошибка!
Исправленное сообщение: [0 0 1 1 1 1 0]
Ошибка не была исправлена корректно.


И для трехкратной ошибки

In [861]:
hamming_correction_test(G, H, syndrome_table, 3, np.array([1, 0, 0, 1]))

Кодовое слово (u): [1 0 0 1]
Отправленное кодовое слово (v): [1 0 0 1 1 0 0]
Допущенная ошибка: [1 1 0 1 0 0 0]
Принятое с ошибкой слово: [0 1 0 0 1 0 0]
Синдром принятого сообщения: [0 1 0]
Обнаружена ошибка!
Исправленное сообщение: [0 1 0 0 1 1 0]
Ошибка не была исправлена корректно.


Результаты аналогичны r = 2. Теперь проведем исследование для r = 4

Пересчитаем H

In [862]:
r = 4
H = generate_hamming_h_matrix(r)
H

array([[1, 1, 1, 1],
       [1, 1, 1, 0],
       [1, 1, 0, 1],
       [1, 1, 0, 0],
       [1, 0, 1, 1],
       [1, 0, 1, 0],
       [1, 0, 0, 1],
       [0, 1, 1, 1],
       [0, 1, 1, 0],
       [0, 1, 0, 1],
       [0, 0, 1, 1],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

Пересчитаем G

In [863]:
G = H_to_G(H, r)
G

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1]])

Заново составим таблицу синдромов

In [864]:
syndrome_table = generate_syndrome_table(H, 1)
syndrome_table

{(1, 1, 1, 1): (0,),
 (1, 1, 1, 0): (1,),
 (1, 1, 0, 1): (2,),
 (1, 1, 0, 0): (3,),
 (1, 0, 1, 1): (4,),
 (1, 0, 1, 0): (5,),
 (1, 0, 0, 1): (6,),
 (0, 1, 1, 1): (7,),
 (0, 1, 1, 0): (8,),
 (0, 1, 0, 1): (9,),
 (0, 0, 1, 1): (10,),
 (1, 0, 0, 0): (11,),
 (0, 1, 0, 0): (12,),
 (0, 0, 1, 0): (13,),
 (0, 0, 0, 1): (14,)}

Передаем сообщение длины 11: "0 0 1 0 1 1 0 0 1 1 1". Допустим однократную ошибку

In [865]:
hamming_correction_test(G, H, syndrome_table, 1, np.array([0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1]))

Кодовое слово (u): [0 0 1 0 1 1 0 0 1 1 1]
Отправленное кодовое слово (v): [0 0 1 0 1 1 0 0 1 1 1 1 1 0 0]
Допущенная ошибка: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
Принятое с ошибкой слово: [0 0 1 0 1 1 0 0 1 1 1 1 1 0 1]
Синдром принятого сообщения: [0 0 0 1]
Обнаружена ошибка!
Исправленное сообщение: [0 0 1 0 1 1 0 0 1 1 1 1 1 0 0]
Ошибка была исправлена успешно!


Теперь допустим двухкратную ошибку

In [866]:
hamming_correction_test(G, H, syndrome_table, 2, np.array([0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1]))

Кодовое слово (u): [0 0 1 0 1 1 0 0 1 1 1]
Отправленное кодовое слово (v): [0 0 1 0 1 1 0 0 1 1 1 1 1 0 0]
Допущенная ошибка: [0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]
Принятое с ошибкой слово: [0 0 1 0 1 0 1 0 1 1 1 1 1 0 0]
Синдром принятого сообщения: [0 0 1 1]
Обнаружена ошибка!
Исправленное сообщение: [0 0 1 0 1 0 1 0 1 1 0 1 1 0 0]
Ошибка не была исправлена корректно.


И, наконец, трехкратную ошибку

In [867]:
hamming_correction_test(G, H, syndrome_table, 3, np.array([0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1]))

Кодовое слово (u): [0 0 1 0 1 1 0 0 1 1 1]
Отправленное кодовое слово (v): [0 0 1 0 1 1 0 0 1 1 1 1 1 0 0]
Допущенная ошибка: [0 0 0 1 0 0 0 0 0 0 1 0 0 0 1]
Принятое с ошибкой слово: [0 0 1 1 1 1 0 0 1 1 0 1 1 0 1]
Синдром принятого сообщения: [1 1 1 0]
Обнаружена ошибка!
Исправленное сообщение: [0 1 1 1 1 1 0 0 1 1 0 1 1 0 1]
Ошибка не была исправлена корректно.


Результаты аналогичны r = 2, r = 3. Код Хэмминга позволяет либо исправлять однократную ошибку, либо обнаруживать двухкратную.

# 3.3

Возьмем r = 2. Проверочную матрицу H* расширенного кода Хэмминга можно построить на основе проверочной матрицы обычного кода Хэмминга

In [868]:
r = 2
H = generate_hamming_h_matrix(r)
H_exp = np.vstack((H, np.array([0] * H.shape[1])))
H_exp = np.hstack((H_exp, np.array([[1] * H_exp.shape[0]]).T))
H_exp

array([[1, 1, 1],
       [1, 0, 1],
       [0, 1, 1],
       [0, 0, 1]])

Порождающую матрицу G* расширенного кода Хэмминга можно построить на основе порождающей матрицы обычного кода Хэмминга. Введем для этого функцию

In [869]:
def expand_G_matrix(G: np.ndarray) -> np.ndarray:
    col = np.zeros((G.shape[0], 1), dtype=int)
    for i in range(G.shape[0]):
        if sum(G[i]) % 2 == 1:
            col[i] = 1
    return np.hstack((G, col))

In [870]:
G = H_to_G(H, r)
G_exp = expand_G_matrix(G)
G_exp

array([[1, 1, 1, 1]])

Построим таблицу синдромов

In [871]:
syndrome_table = generate_syndrome_table(H_exp, 1)
syndrome_table

{(1, 1, 1): (0,), (1, 0, 1): (1,), (0, 1, 1): (2,), (0, 0, 1): (3,)}

# 3.4

Проведем исследование для r = 2. Передаем сообщение длины 1: "1". Допускаем однократную ошибку

In [872]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 1, np.array([1]))

Кодовое слово (u): [1]
Отправленное кодовое слово (v): [1 1 1 1]
Допущенная ошибка: [0 1 0 0]
Принятое с ошибкой слово: [1 0 1 1]
Синдром принятого сообщения: [1 0 1]
Обнаружена ошибка!
Исправленное сообщение: [1 1 1 1]
Ошибка была исправлена успешно!


Допускаем двухкратную ошибку

In [873]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 2, np.array([1]))

Кодовое слово (u): [1]
Отправленное кодовое слово (v): [1 1 1 1]
Допущенная ошибка: [1 1 0 0]
Принятое с ошибкой слово: [0 0 1 1]
Синдром принятого сообщения: [0 1 0]
Обнаружена ошибка!
Синдрома нет в таблице, ошибка не исправлена.


Двухкратную ошибку исправить не удалось, но она была обнаружена

Допускаем трехкратную ошибку

In [874]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 3, np.array([1]))

Кодовое слово (u): [1]
Отправленное кодовое слово (v): [1 1 1 1]
Допущенная ошибка: [0 1 1 1]
Принятое с ошибкой слово: [1 0 0 0]
Синдром принятого сообщения: [1 1 1]
Обнаружена ошибка!
Исправленное сообщение: [0 0 0 0]
Ошибка не была исправлена корректно.


Трехкратную ошибку также удалось обнаружить, но не исправить

Проведем исследование для r = 3. Перепишем H

In [875]:
r = 3
H = generate_hamming_h_matrix(r)
H_exp = np.vstack((H, np.array([0] * H.shape[1])))
H_exp = np.hstack((H_exp, np.array([[1] * H_exp.shape[0]]).T))
H_exp

array([[1, 1, 1, 1],
       [1, 1, 0, 1],
       [1, 0, 1, 1],
       [0, 1, 1, 1],
       [1, 0, 0, 1],
       [0, 1, 0, 1],
       [0, 0, 1, 1],
       [0, 0, 0, 1]])

Заново составим G

In [876]:
G = H_to_G(H, r)
G_exp = expand_G_matrix(G)
G_exp

array([[1, 0, 0, 0, 1, 1, 1, 0],
       [0, 1, 0, 0, 1, 1, 0, 1],
       [0, 0, 1, 0, 1, 0, 1, 1],
       [0, 0, 0, 1, 0, 1, 1, 1]])

Заново заполним таблицу синдромов

In [877]:
syndrome_table = generate_syndrome_table(H_exp, 1)
syndrome_table

{(1, 1, 1, 1): (0,),
 (1, 1, 0, 1): (1,),
 (1, 0, 1, 1): (2,),
 (0, 1, 1, 1): (3,),
 (1, 0, 0, 1): (4,),
 (0, 1, 0, 1): (5,),
 (0, 0, 1, 1): (6,),
 (0, 0, 0, 1): (7,)}

Отсылаем сообщение длины 4: "1 0 1 0". Допустим однократную ошибку

In [878]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 1, np.array([1, 0, 1, 0]))

Кодовое слово (u): [1 0 1 0]
Отправленное кодовое слово (v): [1 0 1 0 0 1 0 1]
Допущенная ошибка: [0 0 1 0 0 0 0 0]
Принятое с ошибкой слово: [1 0 0 0 0 1 0 1]
Синдром принятого сообщения: [1 0 1 1]
Обнаружена ошибка!
Исправленное сообщение: [1 0 1 0 0 1 0 1]
Ошибка была исправлена успешно!


Допустим двухкратную ошибку

In [879]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 2, np.array([1, 0, 1, 0]))

Кодовое слово (u): [1 0 1 0]
Отправленное кодовое слово (v): [1 0 1 0 0 1 0 1]
Допущенная ошибка: [1 0 0 0 0 0 1 0]
Принятое с ошибкой слово: [0 0 1 0 0 1 1 1]
Синдром принятого сообщения: [1 1 0 0]
Обнаружена ошибка!
Синдрома нет в таблице, ошибка не исправлена.


Допустим трехкратную ошибку

In [880]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 3, np.array([1, 0, 1, 0]))

Кодовое слово (u): [1 0 1 0]
Отправленное кодовое слово (v): [1 0 1 0 0 1 0 1]
Допущенная ошибка: [1 1 0 1 0 0 0 0]
Принятое с ошибкой слово: [0 1 1 1 0 1 0 1]
Синдром принятого сообщения: [0 1 0 1]
Обнаружена ошибка!
Исправленное сообщение: [0 1 1 1 0 0 0 1]
Ошибка не была исправлена корректно.


Результаты аналогичны r = 2. Аналогичное исследование проведем для r = 4. Перепишем H

In [881]:
r = 4
H = generate_hamming_h_matrix(r)
H_exp = np.vstack((H, np.array([0] * H.shape[1])))
H_exp = np.hstack((H_exp, np.array([[1] * H_exp.shape[0]]).T))
H_exp

array([[1, 1, 1, 1, 1],
       [1, 1, 1, 0, 1],
       [1, 1, 0, 1, 1],
       [1, 1, 0, 0, 1],
       [1, 0, 1, 1, 1],
       [1, 0, 1, 0, 1],
       [1, 0, 0, 1, 1],
       [0, 1, 1, 1, 1],
       [0, 1, 1, 0, 1],
       [0, 1, 0, 1, 1],
       [0, 0, 1, 1, 1],
       [1, 0, 0, 0, 1],
       [0, 1, 0, 0, 1],
       [0, 0, 1, 0, 1],
       [0, 0, 0, 1, 1],
       [0, 0, 0, 0, 1]])

Заново составим G

In [882]:
G = H_to_G(H, r)
G_exp = expand_G_matrix(G)
G_exp

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1]])

Заново составим таблицу синдромов

In [883]:
syndrome_table = generate_syndrome_table(H_exp, 1)
syndrome_table

{(1, 1, 1, 1, 1): (0,),
 (1, 1, 1, 0, 1): (1,),
 (1, 1, 0, 1, 1): (2,),
 (1, 1, 0, 0, 1): (3,),
 (1, 0, 1, 1, 1): (4,),
 (1, 0, 1, 0, 1): (5,),
 (1, 0, 0, 1, 1): (6,),
 (0, 1, 1, 1, 1): (7,),
 (0, 1, 1, 0, 1): (8,),
 (0, 1, 0, 1, 1): (9,),
 (0, 0, 1, 1, 1): (10,),
 (1, 0, 0, 0, 1): (11,),
 (0, 1, 0, 0, 1): (12,),
 (0, 0, 1, 0, 1): (13,),
 (0, 0, 0, 1, 1): (14,),
 (0, 0, 0, 0, 1): (15,)}

Отсылаем сообщение длины 11: "1 0 0 1 1 1 1 0 1 1 0". Допустим однократную ошибку

In [884]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 1, np.array([1, 0, 0, 1, 1, 1, 1, 0, 1, 1,0]))

Кодовое слово (u): [1 0 0 1 1 1 1 0 1 1 0]
Отправленное кодовое слово (v): [1 0 0 1 1 1 1 0 1 1 0 1 0 0 0 0]
Допущенная ошибка: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Принятое с ошибкой слово: [1 0 0 1 1 1 1 0 1 1 0 1 0 0 1 0]
Синдром принятого сообщения: [0 0 0 1 1]
Обнаружена ошибка!
Исправленное сообщение: [1 0 0 1 1 1 1 0 1 1 0 1 0 0 0 0]
Ошибка была исправлена успешно!


Допустим двухкратную ошибку

In [885]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 2, np.array([1, 0, 0, 1, 1, 1, 1, 0, 1, 1,0]))

Кодовое слово (u): [1 0 0 1 1 1 1 0 1 1 0]
Отправленное кодовое слово (v): [1 0 0 1 1 1 1 0 1 1 0 1 0 0 0 0]
Допущенная ошибка: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0]
Принятое с ошибкой слово: [1 0 1 1 1 1 1 0 1 1 0 1 0 0 1 0]
Синдром принятого сообщения: [1 1 0 0 0]
Обнаружена ошибка!
Синдрома нет в таблице, ошибка не исправлена.


Допустим трехкратную ошибку

In [886]:
hamming_correction_test(G_exp, H_exp, syndrome_table, 3, np.array([1, 0, 0, 1, 1, 1, 1, 0, 1, 1,0]))

Кодовое слово (u): [1 0 0 1 1 1 1 0 1 1 0]
Отправленное кодовое слово (v): [1 0 0 1 1 1 1 0 1 1 0 1 0 0 0 0]
Допущенная ошибка: [0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0]
Принятое с ошибкой слово: [1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 0]
Синдром принятого сообщения: [1 0 1 0 1]
Обнаружена ошибка!
Исправленное сообщение: [1 0 1 1 1 0 1 0 1 1 1 1 1 0 0 0]
Ошибка не была исправлена корректно.


Получили результаты аналогичные r = 2 и r = 3. Расширенный код Хэмминга позволяет исправлять однократные ошибки и обнаруживать двухкратные одновременно.